In [17]:
import pandas as pd
import plotly.graph_objects as go
from utils.helper_functions import *

In [18]:
df = pd.read_csv('datasets/winequalityN.csv')
df.head()

,type,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,white,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,white,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,white,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,white,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,white,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [51]:
cat_cols, num_cols, cat_but_car = grab_col_names(df)

Observations: 6497
Variables: 13
cat_cols: 2
num_cols: 11
cat_but_car: 0
num_but_cat: 1


### Aykırı Gözlemler

In [113]:
summary_outliers = pd.DataFrame()
summary_outliers["col"] = [col for col in num_cols]
summary_outliers["outlier"] = [check_outlier(df, col, 0.05, 0.95) for col in num_cols]   
summary_outliers["low"] = [outlier_thresholds(df, cols, 0.05, 0.95)[0] for cols in num_cols]   
summary_outliers["up"] = [outlier_thresholds(df, cols, 0.05, 0.95)[1] for cols in num_cols]   
describe = df.agg(["min", "max"]).T
describe = pd.DataFrame(describe)
describe = describe.drop("type")
describe = describe.reset_index()
describe.columns = ["col", "min","max"]
describe = pd.merge(describe, summary_outliers, on="col", how="right")
describe

,col,min,max,outlier,low,up
0,fixed acidity,3.8,15.9,False,-0.450000,15.95000
1,volatile acidity,0.08,1.58,True,-0.605000,1.43500
2,citric acid,0.0,1.66,True,-0.715000,1.32500
3,residual sugar,0.6,65.8,True,-19.500000,35.70000
4,chlorides,0.009,0.611,True,-0.083000,0.21300
5,free sulfur dioxide,1.0,289.0,True,-76.500000,143.50000
6,total sulfur dioxide,6.0,440.0,False,-261.500000,486.50000
7,density,0.98711,1.03898,True,0.975662,1.01363
8,pH,2.72,4.01,False,2.175000,4.29500
9,sulphates,0.22,2.0,True,-0.310000,1.45000


In [154]:
fig = go.Figure()
fig.add_trace(go.Bar(
    y=describe["low"],
    x=describe["col"],
    name = "Low Limit",
    orientation="v"
))
fig.add_trace(go.Bar(
    x=describe["col"],
    y=describe["up"],
    name = "Up Limit",
    orientation="v"
))

fig.update_layout(
    title = "Grafik",
    xaxis = dict(title ="Kimyasallar"),
    yaxis = dict(title = "Değerler"),
    barmode="group",
    margin=dict(l=50, r=50, t=50, b=100),
    height=700,
    width=1000
)
fig.show()

### Eksik Gözlemler

In [13]:
isnull = pd.DataFrame()
isnull[0] = df.isnull().sum()
isnull[1] = df.notnull().sum()
isnull["ratio"] = isnull[0] / isnull[1]
isnull.reset_index(inplace=True)


In [15]:
fig = go.Figure(data=[go.Pie(labels=isnull["index"], values=isnull["ratio"])])
fig = go.Figure(data=[go.Pie(labels=isnull["index"], values=isnull[1])])
fig = go.Figure(data=[go.Pie(labels=isnull["index"], values=isnull[0])])
fig.show()

## Kolerasyon

In [28]:
corr_matrix = df.drop("type", axis=1).corr()

In [34]:
fig = go.Figure()
fig.add_trace(go.Heatmap(
    z=corr_matrix.values,
    x=corr_matrix.columns,
    y=corr_matrix.columns,
    colorscale="Reds"
))
fig.show()

1) Hedef değişkenimiz "quality" ile "alcohol"ün güçlü bir ilişkisi var gibi görünüyor ancak bu oran 0.44 grafik en güçlü olduğu için koyu renkle boyamış ama bu çok güçlü bir ilişki mi emin değilim.
2) Sulfur Dioxide'lerin birbirleri arasında güçlü bir kolerasyon var. (0.72)
3) Density ile Alcohol arasında negatif bir güçlü ilişki var (-0.68)

## Quality'nin dağılımı

In [62]:
data = df.groupby("type").agg({"quality": "mean"}).reset_index()
fig = go.Figure()
fig.add_trace(go.Bar(
    x = data["type"],
    y = data["quality"],
    width=0.1,
    marker_color=["red", "lightgreen"]
))
fig.update_layout(
    xaxis = dict(title = "Şarap Türü"),
    yaxis = dict(title = "Kalite Ortalaması")
    
)
fig.show()

Kalite orlamalarında beyaz şarap kırmızı şaraba göre daha yüksek bir orana sahip